In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [3]:
device = torch.device("mps")

In [4]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 170498071/170498071 [00:33<00:00, 5026111.03it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [17]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [18]:
model = ConvNet().to(device)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_step}, loss = {loss.item():.4f}')
            

epoch 1/10, step 2000/12500, loss = 2.2953
epoch 1/10, step 4000/12500, loss = 2.2760
epoch 1/10, step 6000/12500, loss = 2.2760
epoch 1/10, step 8000/12500, loss = 2.2678
epoch 1/10, step 10000/12500, loss = 2.1419
epoch 1/10, step 12000/12500, loss = 2.6886
epoch 2/10, step 2000/12500, loss = 1.6936
epoch 2/10, step 4000/12500, loss = 2.0813
epoch 2/10, step 6000/12500, loss = 2.2486
epoch 2/10, step 8000/12500, loss = 1.8537
epoch 2/10, step 10000/12500, loss = 1.8611
epoch 2/10, step 12000/12500, loss = 1.2876
epoch 3/10, step 2000/12500, loss = 1.7872
epoch 3/10, step 4000/12500, loss = 2.6489
epoch 3/10, step 6000/12500, loss = 1.3625
epoch 3/10, step 8000/12500, loss = 1.2492
epoch 3/10, step 10000/12500, loss = 1.9741
epoch 3/10, step 12000/12500, loss = 2.2344
epoch 4/10, step 2000/12500, loss = 1.7307
epoch 4/10, step 4000/12500, loss = 1.2018
epoch 4/10, step 6000/12500, loss = 0.9763
epoch 4/10, step 8000/12500, loss = 0.8801
epoch 4/10, step 10000/12500, loss = 2.2178
epoc

In [23]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc}%')

accuracy = 58.15
